In [1]:
import os
data_path = os.getcwd()
print(f"{data_path=}")

data_path='/app/notebooks/rapprochements/LVAO'


In [2]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [4]:
# read the excel data
df = pd.read_csv(os.path.join(data_path, 'acteurs.csv'), sep=",",
                 dtype={"code_postal": "string", "adresse": "string", "ville": "string"})

# print the df length
print('The length of the dataframe is: ', len(df))
df.head()

/tmp/ipykernel_541/865948029.py:2: DtypeWarning: Columns (4,24,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(data_path, 'acteurs.csv'), sep=",",


The length of the dataframe is:  286297


,identifiant,paternite,nom,nom_commercial,siren,siret,description,type_dacteur,site_web,telephone,adresse,complement_dadresse,code_postal,ville,latitude,longitude,qualites_et_labels,public_accueilli,reprise,exclusivite_de_reprisereparation,uniquement_sur_rdv,type_de_services,propositions_de_services,emprunter,preter,louer,mettreenlocation,reparer,donner,trier,echanger,revendre,acheter,date_de_derniere_modification
0,222dMMK2fP52fhyhjJcYMY,Longue Vie Aux Objets|ADEME|CRAR Normandie,Repair Café Elbeuf-sur-Seine,NaN,NaN,81794744300021,NaN,ess,https://www.mairie-elbeuf.fr/reouverture-du-re...,NaN,14 rue de la République,Petit Atelier,76500,Elbeuf,49.290368,1.001683,ess,NaN,NaN,NaN,NaN,atelier_pour_reparer_soi_meme,"[{""action"": ""reparer"", ""sous_categories"": [""lu...",NaN,NaN,NaN,NaN,luminaire|materiel hifi et video|materiel info...,NaN,NaN,NaN,NaN,NaN,2024-06-17
1,222xK6YdoHvVNxBzc6L2VM,Longue Vie Aux Objets|ADEME|CITEO,CYCLAD: Point de collecte,NaN,251701900,NaN,NaN,pav_public,NaN,NaN,1 Chemin des Meuniers,NaN,17400,Saint-Julien-De-L'Escap,45.926685,-0.497410,NaN,Particuliers,NaN,NaN,NaN,structure_de_collecte,"[{""action"": ""trier"", ""sous_categories"": [""emba...",NaN,NaN,NaN,NaN,NaN,NaN,emballage_verre,NaN,NaN,NaN,2025-01-13
2,2237TzvCGBM6uM66sScNGu,Longue Vie Aux Objets|ADEME|Bibliothèques - Mi...,Bibliothèque Au Puits Du Livre,NaN,NaN,NaN,NaN,collectivite,NaN,NaN,1 rue des Grémets,NaN,44390,Puceul,47.521964,-1.617676,NaN,NaN,NaN,NaN,NaN,espace_de_partage,"[{""action"": ""emprunter"", ""sous_categories"": [""...","CD, DVD et jeu video|livre",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-10-26
3,2249u3kvjj5kiJvJYF6uTY,Longue Vie Aux Objets|ADEME|CITEO,CACEM: Point de collecte,NaN,249720061,NaN,NaN,pav_public,NaN,NaN,47 Rue Philippe Saint-yves,NaN,97200,Fort-De-France,14.621607,-61.081472,NaN,Particuliers,NaN,NaN,NaN,structure_de_collecte,"[{""action"": ""trier"", ""sous_categories"": [""bout...",NaN,NaN,NaN,NaN,NaN,NaN,autres_emballages_menagers|bouteilles_plastiqu...,NaN,NaN,NaN,2025-03-18
4,224AFbfVE6DagFkZkBh5Q3,Longue Vie Aux Objets|ADEME|Longue Vie Aux Objets,Les Sens Du Bois,NaN,NaN,82759528100011,NaN,artisan,NaN,NaN,894 ROUTE DE CASSIS,QUARTIER LES FOURNIERS,13830,ROQUEFORT LA BEDOULE,43.247187,5.573553,NaN,NaN,NaN,NaN,NaN,service_de_reparation,"[{""action"": ""reparer"", ""sous_categories"": [""Si...",NaN,NaN,NaN,NaN,accessoire et meuble de jardin|decoration|joue...,NaN,NaN,NaN,NaN,NaN,2024-07-18


In [5]:
# display df columns
print(df.columns)

Index(['identifiant', 'paternite', 'nom', 'nom_commercial', 'siren', 'siret',
       'description', 'type_dacteur', 'site_web', 'telephone', 'adresse',
       'complement_dadresse', 'code_postal', 'ville', 'latitude', 'longitude',
       'qualites_et_labels', 'public_accueilli', 'reprise',
       'exclusivite_de_reprisereparation', 'uniquement_sur_rdv',
       'type_de_services', 'propositions_de_services', 'emprunter', 'preter',
       'louer', 'mettreenlocation', 'reparer', 'donner', 'trier', 'echanger',
       'revendre', 'acheter', 'date_de_derniere_modification'],
      dtype='object')


# map to inputs

In [6]:
def should_include_row(df_row_raw):
    df_row = dict(df_row_raw)
    paternite = df_row["paternite"].lower()
    
    return not (("citeo" in paternite) or ("refashion" in paternite))

filtered_df = df[df.apply(should_include_row, axis=1)]

In [7]:
print(f"{len(filtered_df)=}")

len(filtered_df)=145676


In [8]:
def row_to_input(df_row_raw):
    df_row = dict(df_row_raw)
    lat = df_row["latitude"]
    lng = df_row["longitude"]
    if pd.isna(df_row["adresse"]):
        address = None
    else:
        address = df_row["adresse"] + ", " + df_row["code_postal"] + " " + df_row["ville"]
        
    return {
        "ext_id" : df_row["identifiant"],
        "name": df_row["nom"],
        "address": address,
        "lat": float(lat),
        "lng": float(lng)
    }


inputs = list(filtered_df.apply(row_to_input, axis=1))

In [9]:
import random
sample_inputs = random.sample(inputs, 200)
print(sample_inputs)

[{'ext_id': 'R7WP4r64yDnWXB5gZuWjsh', 'name': 'DECHETTERIE DE FEUQUIERES - CC DE LA PICARDIE VERTE', 'address': 'ZONE INDUSTRIELLE DU MOULIN RENARD, 60960 FEUQUIERES', 'lat': 49.652396, 'lng': 1.857742}, {'ext_id': 'FtGwcAtYiNaWERuvxUoZot', 'name': 'TPH VIENNE', 'address': '17  COURS ROMESTANG, 38200 Vienne', 'lat': 45.523311, 'lng': 4.874457}, {'ext_id': 'Z5QtSgMYh4jFwDUm9mJjaU', 'name': 'Decathlon Essentiel Caudry', 'address': 'Boulevard du 8 mai 1945, 59540 Caudry', 'lat': 50.125182, 'lng': 3.422667}, {'ext_id': 'ioF6dBgtdPz8UduDZkuQK2', 'name': 'AGRI MONT JURA CHAMPAGNOLE', 'address': '528 RUE DU MARECHAL DE LATTRE DE TASSIGNY, 39300 Champagnole', 'lat': 46.7472921, 'lng': 5.9211972}, {'ext_id': 'ehN7ifcJhrL3J4iewehwUr', 'name': 'Melody Rose', 'address': '882 RUE FREDERIC MISTRAL, 13370 MALLEMORT', 'lat': 43.726287, 'lng': 5.182174}, {'ext_id': 'dkhsVcHmWYzJgYfuq5wvC2', 'name': 'GGE SAINT CYR CENTRE AUTO', 'address': 'Chemin des Pradeaux, 83270 Saint-Cyr-sur-Mer', 'lat': 43.183766,

In [10]:
from batid.services.guess_bdg_new import Guesser, GeocodeNameHandler, GeocodeAddressHandler, ClosestFromPointHandler
guesses_path = "./guesses.json"
guesser = Guesser(batch_size=90)
guesser.handlers = [
    GeocodeNameHandler(bbox_apothem_in_meters = 5000),
    GeocodeAddressHandler(),
    ClosestFromPointHandler()
]
guesser.create_work_file(list(sample_inputs), guesses_path)
guesser.guess_work_file(guesses_path)

  0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
guesser = Guesser()
guesser.load_work_file(guesses_path)
guesser.report()

-- Report --
Number of rows: 200
Number of match: 152 (76.00%)

-- finished_steps --
Rows with finished_steps closest_from_point: 200 (100.00%)
Rows with finished_steps geocode_address: 200 (100.00%)
Rows with finished_steps geocode_name: 200 (100.00%)
Rows with empty finished_steps: 0 (0.00%)

-- match_reasons : absolute --
match_reason
found_name_in_osm_point_on_bdg            75
precise_address_match                     42
point_on_bdg                              22
isolated_closest_bdg                      11
found_name_in_osm_isolated_closest_bdg     2
Name: count, dtype: int64

-- match_reasons : % --
match_reason
found_name_in_osm_point_on_bdg            37.5
precise_address_match                     21.0
point_on_bdg                              11.0
isolated_closest_bdg                       5.5
found_name_in_osm_isolated_closest_bdg     1.0
Name: count, dtype: float64

-- Inputs --
rows with ban_id: 47 (23.50%)


In [13]:
guesser.to_csv(os.path.join(data_path, 'results.csv'), ext_id_col_name="ext_id")